In [8]:
%pip install -r requirements.txt --upgrade
%pip install miditok

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [9]:
import json
from pathlib import Path
from miditok import REMI, TokSequence
# from copy import deepcopy
import pandas as pd
import numpy as np
import gpt_mini.midi_encoder as midichar
import pickle
from gpt_mini.config import DEFAULT_DEVICE, CONFIG

In [10]:
df = pd.read_csv(CONFIG["preprocess"]["new_dataset_index"], delimiter="\t", header=None, names=['data'])
display(df.head())

,data
0,drums/1.mid
1,drums/2.mid
2,drums/3.mid
3,drums/4.mid
4,drums/5.mid


In [11]:
# for index, row in df.iterrows():
#     sample = np.array( json.loads(row['data']) , dtype=np.uint32 )
#     try:
#         midichar.decode_midi(notes=sample,
#                         out_file=f"drums/{index}.mid",
#                         instrument_name="Standard Kit")
#     except Exception as e:
#         pass


## Train The Tokenizer

In [12]:
# using defaults parameters (constants.py)
tokenizer = REMI()
# paths_midis = list(Path("drums").glob('**/*.mid'))

tokenizer.train(
    vocab_size=CONFIG["tokenizer"]["vocab_size"],
    files_paths=list(df["data"]))
with open(CONFIG["tokenizer"]["model"], 'wb') as f:
    pickle.dump(tokenizer, f)


## Test The Tokenizer

In [13]:
tokenizer = None
with open(CONFIG["tokenizer"]["model"], 'rb') as f:
    tokenizer = pickle.load(f)
midi_path = 'drums/10.mid'
tokens = tokenizer.encode(midi_path)

# tokens_with_bpe = tokenizer.encode_token_ids(deepcopy(tokens))         # copy as the method is inplace
# tokens_no_bpe = tokenizer.decode_token_ids(deepcopy(tokens_with_bpe))
# print(tokens_with_bpe, tokens_no_bpe)

print(tokens[0].ids)

[283, 1456, 425, 933, 27061, 693, 7510, 10883, 766, 24218, 32123, 10883, 22260, 3047, 410, 766, 3834, 283, 10883, 685, 24218, 32123, 32123, 32123, 32123, 32123, 32123, 32123, 10883, 766, 24218, 32123, 10883, 20287]


## Split into Training And Test Sets

In [14]:
df_shuffled = df.sample(frac=1, random_state=42)

train_ratio = 0.9
train_index = int(len(df_shuffled) * train_ratio)
df_train = df_shuffled[:train_index]
df_test = df_shuffled[train_index:]

df_train.to_csv(CONFIG["preprocess"]["data_train"], sep="\t", header=None, index=False)
df_test.to_csv(CONFIG["preprocess"]["data_validation"], sep="\t", header=None, index=False)
